In [2]:
# iris data
# sklearn.tree dt_clf
# cross val score
# numpy, pandas

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, train_test_split
import numpy as np
import pandas as pd

In [4]:
iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris_data.data # X값
target = iris_data.target # y값

In [6]:
#성능 지표는 정확도(accuracy), 교차 검증 세트는 3개
scores = cross_val_score(dt_clf, data, target, scoring='accuracy', cv=3)
print('cv별 정확도:', np.round(scores,4))
print('평균 cv 정확도:', np.round(np.mean(scores),4))

cv별 정확도: [0.98 0.94 0.98]
평균 cv 정확도: 0.9667


## grid_search_cv 학습

교차검증과 최적 하이퍼파라미터튜닝을 한 번에 하게 해주는 GridSearchCV API를 활용 

In [9]:
?DecisionTreeClassifier

GridSearchCV는 교차 검증을 기반으로 이 하이퍼 파라미터의 최적 값을 찾게 해준다. 즉, 데이터 세트를 cross_validation을 위한 학습/테스트 세트로 자동으로 분할한 뒤에 하이퍼 파라미터 그리드에 기술된 모든 파라미터를 순차적으로 적용해 최적의 파라미터를 찾을 수 있게 해준다.

GridSearchCV는 사용자가 튜닝하고자 하는 여러 종류의 하이퍼 파라미터를 다양하게 테스트하면서 최적의 파라미터를 편리하게 찾게 해주지만 동시에 순차적으로 파라미터를 테스트하므로 수행시간이 상대적으로 오래 걸리는 것에 유념해야 한다. 

만약에 CV가 3회라면 개별 파라미터 조합마다 3개의 폴딩 세트를 3회에 걸쳐 학습/평가해 평균값으로 성능을 측정한다. 6개의 파라미터 조합이라면 총 CV 3회 X 6개 파라미터 조합 = 18회의 학습/평가가 이뤄진다. 

 GridSearchCV 클래스의 생성자로 들어가는 주요 파라미터는 다음과 같다.

* estimator: classifier, regressor, pipeline이 사용될 수 있다.


* param_grid: key + 리스트 값을 가지는 딕셔너리가 주어진다. 


* scoring: 예측 성능을 측정할 평가 방법을 지정한다. 보통은 사이킷런의 성능 평가 지표를 지정하는 문자열(예:정확도의 경우 'accuracy')로 지정하나 별도의 성능 평가 지표 함수도 지정할 수 있다.


* cv:교차 검증을 위해 분할되는 학습/테스트 세트의 개수를 지정한다.


* refit: 디폴트가 True이며 True로 생성 시 가장 최적의 하이퍼 파라미터를 찾은 뒤 입력된 estimator 객체를 해당 하이퍼 파라미터로 재학습시킨다.

간단한 예제를 통해서 GridSearchCV API의 사용법을 익혀보자. 결정 트리 알고리즘의 여러가지 최적화 파라미터를 순차적으로 적용해 붓꽃 데이터를 예측 분석하는데 GridSearchCV를 이용하도록 하겠다. train_test_split()을 이용해 학습 데이터와 테스트 데이터를 먼저 분리하고 학습데이터에서 GridSearchCV를 이용해 최적 하이퍼 파라미터를 추출해보자. 결정 트리 알고리즘을 구현한 DecisionTreeClassifier의 중요 하이퍼 파라미터인 max_depth와 min_samples_split의 값을 변화시키면서 최적화를 진행하겠다. 테스트할 하이퍼 파라미터 세트는 딕셔너리 형태로 하이퍼 파라미터의 명칭은 문자열 Key 값으로, 하이퍼 파라미터의 값은 리스트 형으로 설정한다.   

In [8]:
## 전체 iris 데이터를 분할하여 가져가보도록 하자
from sklearn.datasets import load_iris # 데이터 
from sklearn.tree import DecisionTreeClassifier # 알고리즘
from sklearn.model_selection import train_test_split # 데이터 분할
from sklearn.model_selection import GridSearchCV # CV 및 하이퍼파라미터 튜닝 
from sklearn.metrics import accuracy_score # 평가지표

import numpy as np
import pandas as pd

In [10]:
# 데이터를 재로딩 및 학습과 테스트 데이터 분리
iris = load_iris() # 전체의 데이터

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target,
                                                   test_size=0.2, random_state=121)

dt_clf = DecisionTreeClassifier() # 의사결정나무 객체화 

### 파라미터를 딕셔너리 형태로 설정
parameters = {'max_depth':[1,2,3],
             'min_samples_split':[2,3]}

In [22]:
grid_dtree= GridSearchCV(dt_clf, param_grid=parameters, cv=3, refit=True)
    
# iris학습데이터를 위의 grid_dtree로 학습
grid_dtree.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [1, 2, 3], 'min_samples_split': [2, 3]})

In [23]:
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'split0_test_score','split1_test_score','split2_test_score',
          'mean_test_score','rank_test_score']]

,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,rank_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700,0.7,0.70,0.700000,5
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700,0.7,0.70,0.700000,5
2,"{'max_depth': 2, 'min_samples_split': 2}",0.925,1.0,0.95,0.958333,3
3,"{'max_depth': 2, 'min_samples_split': 3}",0.925,1.0,0.95,0.958333,3
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975,1.0,0.95,0.975000,1
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975,1.0,0.95,0.975000,1


In [25]:
# validation 했을 때의 정확도

print('GridSearchCV 최적 파라미터:', grid_dtree.best_params_)
print('GridSearchCV 최고 정확도:', grid_dtree.best_score_)

GridSearchCV 최적 파라미터: {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.975


In [28]:
# GridSearchCV를 통해 확인한 최적의 파라미터

result_estimator = grid_dtree.best_estimator_

# GridSearchCV의 최적의 best_estimator가 이미 학습이 되어있으므로
# 별도의 학습은 필요가 없음

result_pred = result_estimator.predict(X_test)
print('test(평가) 데이터 세트 정확도:', np.round(accuracy_score(y_test,result_pred),4))

test(평가) 데이터 세트 정확도: 0.9667
